# To Do
- eliminar as colunas claramente irrelevantes para a afinidade

In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

## Filtrando colunas

In [2]:
df_raw = pd.read_csv("../dados/inputs/microdados_ed_basica_2024.csv", sep=";", encoding="latin1")

print(df_raw.shape)

# é  um df muito grande (215_545, 426), primeiro passo: filtrar as colunas

/var/folders/c6/vss0yj1j115_3ykhy6lb4kcw0000gn/T/ipykernel_45764/1816987993.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("../dados/inputs/microdados_ed_basica_2024.csv", sep=";", encoding="latin1")


(215545, 426)


In [3]:
df_filtro = pd.read_excel("../dados/temporarios/filtro_colunas.xlsx", skiprows=8, usecols=[1, 2])

df_filtro = df_filtro[~df_filtro["util"].isin(["DESCARTAR", "TALVEZ", "AUSENTE", np.nan])]

lista_cols = df_filtro["Nome da Variável"].astype(str).to_list()

print(f"Reduzimos para {len(lista_cols)} colunas")

with open("../dados/banco_dados/colunas_relevantes_md_edb.json", "w", encoding="utf-8") as f:
    json.dump(lista_cols, f, ensure_ascii=False, indent=4)
    

Reduzimos para 63 colunas


In [4]:

with open("../dados/banco_dados/colunas_relevantes_md_edb.json", "r", encoding="utf-8") as f:
    cols_interesse = json.load(f)

df_filtrado = df_raw[cols_interesse]

df_filtrado.shape


(215545, 63)

## Combinando colunas

In [5]:
def soma_colunas(df: pd.DataFrame, colunas: list[str], novo_nome: str, drop = True):
    """
    df: DataFrame
    colunas: lista com nomes de colunas a serem somadas
    novo_nome: nome da nova coluna criada
    """
    df[novo_nome] = df[colunas].sum(axis=1).astype(int)
    if drop:
        df = df.drop(columns=colunas)
        
    return df



In [6]:

df = df_filtrado.copy()


infraestrutura = ["IN_ESGOTO_REDE_PUBLICA", "IN_ENERGIA_REDE_PUBLICA",
                  "IN_LIXO_SERVICO_COLETA"]
df = soma_colunas(df, infraestrutura, "infraestrutura")



estrutura_pobre = ["IN_TERREIRAO", "IN_ACESSIBILIDADE_INEXISTENTE"]
df = soma_colunas(df, estrutura_pobre, "estrutura_pobre")

estrutura_basica = ["IN_ALMOXARIFADO", "IN_BIBLIOTECA", "IN_LABORATORIO_INFORMATICA",
                    "IN_PATIO_DESCOBERTO", "IN_QUADRA_ESPORTES_DESCOBERTA"]
df = soma_colunas(df, estrutura_basica, "estrutura_basica")

estrutura_padrao = ["IN_AUDITORIO", "IN_BIBLIOTECA_SALA_LEITURA",
                    "IN_LABORATORIO_CIENCIAS", "IN_PATIO_COBERTO"]
df = soma_colunas(df, estrutura_padrao, "estrutura_padrao")

estrutura_premium = ["IN_AREA_PLANTIO", "IN_BANHEIRO_CHUVEIRO", "IN_PISCINA", "IN_SALA_ATELIE_ARTES",
                     "IN_SALA_MUSICA_CORAL", "IN_SALA_ESTUDIO_DANCA", "IN_SALA_ESTUDIO_GRAVACAO",
                     "IN_SALA_REPOUSO_ALUNO", "IN_ACESSIBILIDADE_ELEVADOR", "IN_MATERIAL_PED_MUSICAL"]
df = soma_colunas(df, estrutura_premium, "estrutura_premium")



ativos_basico = ["QT_EQUIP_TV", "QT_EQUIP_MULTIMIDIA", "QT_DESKTOP_ALUNO"]
df = soma_colunas(df, ativos_basico, "qt_ativos_basico")

ativos_premium = ["QT_EQUIP_LOUSA_DIGITAL", "QT_COMP_PORTATIL_ALUNO", "QT_TABLET_ALUNO"]
df = soma_colunas(df, ativos_premium, "qt_ativos_premium")



pessoal_basico = ["QT_PROF_ADMINISTRATIVOS", "QT_PROF_SERVICOS_GERAIS",
                  "QT_PROF_BIBLIOTECARIO", "QT_PROF_ALIMENTACAO", "QT_PROF_SECRETARIO"]
df = soma_colunas(df, pessoal_basico, "pessoal_basico")

pessoal_padrao = ["QT_PROF_SAUDE", "QT_PROF_COORDENADOR", "QT_PROF_PEDAGOGIA", "QT_PROF_MONITORES",
                "QT_PROF_GESTAO", "QT_PROF_ASSIST_SOCIAL"]
df = soma_colunas(df, pessoal_padrao, "pessoal_padrao")

pessoal_premium = ["QT_PROF_FONAUDIOLOGO", "QT_PROF_NUTRICIONISTA", "QT_PROF_PSICOLOGO",
               "QT_PROF_SEGURANCA", "QT_PROF_AGRICOLA"]
df = soma_colunas(df, pessoal_premium, "pessoal_premium")



alunos = ["QT_MAT_INF", "QT_MAT_FUND_AI", "QT_MAT_FUND_AF", "QT_MAT_MED"]
df = soma_colunas(df, alunos, "qt_alunos", drop=False)


professores = ["QT_DOC_INF", "QT_DOC_FUND_AI", "QT_DOC_FUND_AF", "QT_DOC_MED"]
df_combinado = soma_colunas(df, professores, "qt_professores")


df_combinado.shape

(215545, 25)

In [7]:
df_combinado["tipo_ocupacao"] = (df_combinado["TP_OCUPACAO_PREDIO_ESCOLAR"]
                                 .fillna(2)
                                 .replace({1:3, 2:1, 3:2}))  # 1 alugado, 2 cedido, 3 proprio


df_combinado = df_combinado.drop(columns=["TP_OCUPACAO_PREDIO_ESCOLAR"])

In [8]:
df_combinado.head(20)

# CO_ENTIDADE
# TP_DEPENDENCIA
# TP_CATEGORIA_ESCOLA_PRIVADA
# CO_CEP
# TP_SITUACAO_FUNCIONAMENTO

# IN_MEDIACAO_PRESENCIAL


,CO_ENTIDADE,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,CO_CEP,TP_SITUACAO_FUNCIONAMENTO,QT_SALAS_UTILIZADAS,IN_EXAME_SELECAO,IN_MEDIACAO_PRESENCIAL,QT_MAT_INF,QT_MAT_FUND_AI,QT_MAT_FUND_AF,QT_MAT_MED,infraestrutura,estrutura_pobre,estrutura_basica,estrutura_padrao,estrutura_premium,qt_ativos_basico,qt_ativos_premium,pessoal_basico,pessoal_padrao,pessoal_premium,qt_alunos,qt_professores,tipo_ocupacao
0,11022558,2,NaN,76954000,1,1.0,0.0,1.0,0.0,4.0,0.0,0.0,0,2,0,0,0,0,1,0,0,0,4,1,3.0
1,11024275,2,NaN,76954000,1,13.0,0.0,1.0,0.0,0.0,0.0,0.0,2,0,2,2,1,34,6,9,6,0,0,0,3.0
2,11024291,3,NaN,76954000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1.0
3,11024666,3,NaN,76954000,1,9.0,0.0,1.0,32.0,66.0,71.0,0.0,1,1,0,1,1,2,0,5,1,2,169,13,3.0
4,11024682,2,NaN,76954000,1,33.0,0.0,1.0,0.0,0.0,237.0,361.0,2,0,4,2,2,45,0,35,13,0,598,51,3.0
5,11024917,3,NaN,76954000,1,6.0,0.0,1.0,13.0,24.0,20.0,0.0,1,1,2,2,1,2,0,5,0,0,57,11,3.0
6,11024968,2,NaN,76954000,1,16.0,0.0,1.0,0.0,0.0,117.0,250.0,2,0,4,4,2,36,19,17,4,4,367,33,3.0
7,11025034,3,NaN,76954000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1.0
8,11025115,3,NaN,76954000,1,1.0,0.0,1.0,0.0,20.0,0.0,0.0,1,1,1,1,0,1,0,0,0,0,20,1,3.0
9,11025310,3,NaN,76954000,1,11.0,0.0,1.0,39.0,95.0,104.0,0.0,1,1,2,2,1,2,1,11,1,0,238,18,3.0


## Salvando resultado

In [9]:
df_combinado.to_csv("../dados/temporarios/01-md_edb_colunas.csv", index=False)